In [ ]:
import random
import pandas as pd
import numpy as np
import os
#import matplotlib.pyplot as plt
from tqdm import tqdm
import re
import gc
#from sklearn.cluster import DBSCAN
#from scipy.spatial import distance
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#from sklearn.cluster import KMeans
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score
)
import optuna
import joblib
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from optuna.samplers import TPESampler
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
#from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score
import optuna
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore", message="Using categorical_feature in Dataset.")
import warnings
warnings.filterwarnings("ignore", message="Cannot compute class probabilities")
import os, re, glob, json
import numpy as np
import pandas as pd
import cloudpickle  

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import glob
import cloudpickle
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing

# ─────────────────────────────────────────────
# ✨ 병렬 프로세스 수 설정
# - None  → 최대 코어 사용
# - 정수값 → 그 수만큼 제한
# ─────────────────────────────────────────────
n_jobs = None  # 예: None, 8, 4 등

if n_jobs is None:
    n_jobs = multiprocessing.cpu_count()
print(f"✅ 병렬 프로세스 수: {n_jobs}")

# ─────────────────────────────────────────────
# 데이터 로딩
# ─────────────────────────────────────────────
train = pd.read_parquet("train_preprocess_Final.parquet")
test  = pd.read_parquet("test_preprocess_Final.parquet")
submit = pd.read_csv("sample_submission.csv")

cat_list = train.select_dtypes(include=['category', 'object']).columns.tolist()
_ = cat_list
X_test = test.copy()

# ─────────────────────────────────────────────
# 모델 파일 로딩
# ─────────────────────────────────────────────
MODEL_DIR = "Weights"
pattern = os.path.join(MODEL_DIR, "lgbm_seed*_model*_fold*.pkl")
pkl_paths = glob.glob(pattern)
if not pkl_paths:
    raise FileNotFoundError(f"모델 파일을 찾을 수 없습니다: {pattern}")

def parse_ids(path):
    m = re.search(r"seed(\d+)_model(\d+)_fold(\d+)\.pkl", os.path.basename(path))
    if not m:
        return (999999, 999999, 999999)
    return tuple(int(g) for g in m.groups())

pkl_paths = sorted(pkl_paths, key=parse_ids)

# ─────────────────────────────────────────────
# 예측 함수
# ─────────────────────────────────────────────
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def predict_model(path):
    seed, model_idx, fold_num = parse_ids(path)
    print(f"[PID:{os.getpid()}] Predicting → model_idx: {model_idx}, fold: {fold_num}")
    with open(path, "rb") as f:
        model = cloudpickle.load(f)
    raw = model.predict(X_test, raw_score=True)
    prob = sigmoid(raw)
    return seed, prob

# ─────────────────────────────────────────────
# 병렬 처리
# ─────────────────────────────────────────────
seed_to_preds = defaultdict(list)

with ProcessPoolExecutor(max_workers=n_jobs) as executor:
    future_to_path = {executor.submit(predict_model, path): path for path in pkl_paths}
    for future in as_completed(future_to_path):
        seed, prob = future.result()
        seed_to_preds[seed].append(prob)

# ─────────────────────────────────────────────
# 평균 및 저장
# ─────────────────────────────────────────────
seed_averages = [np.mean(preds_list, axis=0) for seed, preds_list in sorted(seed_to_preds.items())]
final_pred = np.mean(seed_averages, axis=0)

submit["clicked"] = final_pred
submit.to_csv("Inference.csv", index=False)
